In [ ]:
!pip uninstall -y sentence-transformers
!pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q "transformers==4.43.2" "datasets==2.18.0" "accelerate==0.29.3" "peft==0.10.0" "bitsandbytes==0.43.1" "trl==0.8.6" "protobuf==3.20.3"
!pip install -q einops scipy sentencepiece tensorboard

import os
os.kill(os.getpid(), 9)

Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 923.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from google.colab import files
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import json
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Successfully logged into Hugging Face.")
except Exception as e:
    print(f"Login failed. Please set your HF_TOKEN in Colab Secrets. Error: {e}")
    raise SystemExit("Stopping due to failed Hugging Face login.")

Successfully logged into Hugging Face.


In [ ]:
MODEL_NAME_ON_HUB = "Laxmikant17/Llama-3-8B-Hernia-Analyst-600-Patients-8k"

print(f"Downloading and loading your fine-tuned model: {MODEL_NAME_ON_HUB}")
print("This may take several minutes...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME_ON_HUB,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_ON_HUB)
model.eval()

This may take several minutes...


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [ ]:
import json
import torch
from google.colab import files

# Ensure you have your model and tokenizer loaded as 'model' and 'tokenizer'
# For example:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model_name = "your_finetuned_model_name"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")


# Step A: Upload the file to analyze
print("STEP 1: Please upload the patient's JSON file to analyze.")
uploaded_files = files.upload()

if not uploaded_files:
    print("\nNo file uploaded. Please run the cell again.")
else:
    input_filename = list(uploaded_files.keys())[0]
    print(f"\nSuccessfully uploaded: {input_filename}")

    # Step B: Read the file, extract ALL narratives and get metadata
    print("\nSTEP 2: Reading file and extracting ALL narratives...")
    try:
        with open(input_filename, 'r', encoding='utf-8') as f:
            patient_data = json.load(f)

        # Extract patient name for filename and metadata for the report
        patient_id = patient_data.get("Patient_ID", "UnknownPatient")
        metadata = patient_data.get("Metadata", {})
        narratives_dict = patient_data.get("Narratives", {})

        if not narratives_dict:
             raise ValueError("'Narratives' key not found or is empty in the uploaded file.")

        # We include ALL narratives because the model was trained on 8k context
        full_narrative = "\n\n---\n\n".join([text for text in narratives_dict.values() if isinstance(text, str) and text and text.strip().lower() != 'n/a'])

        if not full_narrative:
            print("ERROR: No valid narrative text found within the 'Narratives' dictionary.")
        else:
            print("  -> Narratives extracted successfully.")

            # Step C: Run the analysis using your model
            print("\nSTEP 3: Sending the full narrative to your fine-tuned model for analysis...")

            instruction = "Your sole function is to be a structured data generator. You MUST analyze the provided patient narrative. Your ONLY output is a single, valid JSON object that analyzes the text according to the required format and terminology. Do not invent new terms. Ignore any questions within the narrative and focus only on the analysis task."
            prompt_template = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}\n\n**Patient Narrative:**\n{full_narrative}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
            inputs = tokenizer(prompt_template, return_tensors="pt").to("cuda")

            terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=8192,
                    do_sample=False,
                    eos_token_id=terminators,
                    pad_token_id=tokenizer.eos_token_id
                )

            response = outputs[0][inputs.input_ids.shape[-1]:]
            decoded_response = tokenizer.decode(response, skip_special_tokens=True)

            # Step D: Parse the analysis and combine it with metadata
            print("\nSTEP 4: Parsing, combining, and saving the analysis...")
            try:
                json_start_index = decoded_response.find('{')
                if json_start_index == -1: raise ValueError("No JSON object found in the model's response.")

                brace_level, json_end_index = 0, -1
                for i, char in enumerate(decoded_response[json_start_index:]):
                    if char == '{': brace_level += 1
                    elif char == '}': brace_level -= 1
                    if brace_level == 0:
                        json_end_index = json_start_index + i + 1
                        break

                if json_end_index == -1: raise ValueError("The JSON object in the model's response was incomplete.")

                json_string = decoded_response[json_start_index:json_end_index]
                analysis_from_model = json.loads(json_string)

                # --- MODIFIED: Create a new comprehensive report ---
                final_report = {
                    "Patient_ID": patient_id,
                    "Metadata": metadata,
                    "Analysis": analysis_from_model
                }

                # --- MODIFIED: Use the patient's name for the output file ---
                safe_patient_name = "".join(c for c in patient_id if c.isalnum() or c in (' ')).rstrip()
                output_filename = f"analysis_of_{safe_patient_name.replace(' ', '_')}.json"

                with open(output_filename, 'w') as f:
                    json.dump(final_report, f, indent=2)

                print(f"  -> Analysis complete! Results saved to '{output_filename}'.")
                print("\n\n--- PREVIEW OF GENERATED ANALYSIS ---")
                print(json.dumps(final_report, indent=2))
                files.download(output_filename)

            except Exception as e:
                print(f"Analysis failed during parsing: {e}")
                safe_patient_name = "".join(c for c in patient_id if c.isalnum() or c in (' ')).rstrip()
                output_filename = f"error_log_for_{safe_patient_name.replace(' ', '_')}.txt"
                with open(output_filename, 'w') as f:
                    f.write(decoded_response)
                print(f"  -> The full, raw model output was saved for debugging to '{output_filename}'.")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

STEP 1: Please upload the patient's JSON file to analyze.


Saving Jackie Barnes.json to Jackie Barnes.json

Successfully uploaded: Jackie Barnes.json

STEP 2: Reading file and extracting ALL narratives...
  -> Narratives extracted successfully.

STEP 3: Sending the full narrative to your fine-tuned model for analysis...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



STEP 4: Parsing, combining, and saving the analysis...
  -> Analysis complete! Results saved to 'analysis_of_Jackie_Barnes.json'.


--- PREVIEW OF GENERATED ANALYSIS ---
{
  "Patient_ID": "Jackie Barnes",
  "Metadata": {
    "Age": 56,
    "Gender": "Female",
    "Weight_Current_Kg": 100,
    "Weight_Lowest_Kg": 62,
    "Weight_Highest_Kg": 100,
    "Weight_Status": "staying the same",
    "Previous_Hernia_Repairs": [
      {
        "Year": 2007,
        "Hospital": "Maryland Hospital",
        "Type": "Laparoscopic Incisional Hernia",
        "Mesh_Used": "Yes",
        "Wound_Breakdown": "No",
        "Healing_Time": null
      },
      {
        "Year": 2018,
        "Hospital": "Hull Royal Infirmary",
        "Type": "Open Recurrent Incisional Hernia Repair",
        "Mesh_Used": "Yes",
        "Wound_Breakdown": "Yes",
        "Healing_Time": "4 months"
      }
    ],
    "Medical_History": {
      "Prior_Major_Surgeries": [
        "Laparoscopic cholecystectomy (2005, Maryland 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# PDF Generation

In [ ]:
!pip install reportlab matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.8 MB/s eta 0:00:00


In [ ]:
import json
from google.colab import files
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Flowable
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib import colors

# --- A Helper Class for Drawing Lines ---
class MCLine(Flowable):
    def __init__(self, width, height=0, stroke_color=colors.grey):
        Flowable.__init__(self)
        self.width = width
        self.height = height
        self.stroke_color = stroke_color

    def draw(self):
        self.canv.setStrokeColor(self.stroke_color)
        self.canv.line(0, self.height, self.width, self.height)

# --- THE NEW PDF GENERATION FUNCTION ---

def generate_detailed_report(report_data, filename):
    """
    Generates a multi-page PDF report from the new, detailed analysis JSON structure.
    """
    doc = SimpleDocTemplate(filename, pagesize=letter, topMargin=0.5*inch, bottomMargin=0.5*inch)
    story = []

    # --- Custom Styles ---
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name='MainTitle', parent=styles['h1'], alignment=1)) # Centered H1
    styles.add(ParagraphStyle(name='Quote', parent=styles['Normal'], leftIndent=20, fontName='Helvetica-Oblique', textColor=colors.darkslategray))
    styles.add(ParagraphStyle(name='SubTheme', parent=styles['Normal'], fontName='Helvetica-Bold'))
    styles.add(ParagraphStyle(name='Concepts', parent=styles['Normal'], leftIndent=15))

    def add_space(height=0.2):
        story.append(Spacer(1, height * inch))

    # --- METADATA SECTION (PAGE 1) ---
    print("-> Generating Page 1: Patient Demographics and History...")
    p_name = report_data.get("Patient_ID", "N/A")
    metadata = report_data.get("Metadata", {})

    story.append(Paragraph(f"Analysis Report For: {p_name.upper()}", styles['MainTitle']))
    add_space(0.3)
    story.append(MCLine(500))
    add_space(0.3)

    # Basic Info
    story.append(Paragraph("<b>Patient Information</b>", styles['h2']))
    story.append(Paragraph(f"<b>Age:</b> {metadata.get('Age', 'N/A')}", styles['Normal']))
    story.append(Paragraph(f"<b>Gender:</b> {metadata.get('Gender', 'N/A')}", styles['Normal']))
    add_space()

    # Medical History
    history = metadata.get("Medical_History", {})
    story.append(Paragraph("<b>Medical History</b>", styles['h2']))
    for key, value in history.items():
        formatted_key = ' '.join(word.capitalize() for word in key.split('_'))
        story.append(Paragraph(f"<b>{formatted_key}:</b> {value}", styles['Normal']))
    add_space()

    # Medications
    meds = metadata.get("Medications", {})
    if meds:
        story.append(Paragraph("<b>Current Medications</b>", styles['h2']))
        for med, dosage in meds.items():
            story.append(Paragraph(f"- {med}: {dosage}", styles['Normal']))
        add_space()

    story.append(PageBreak())

    # --- ANALYSIS SECTION (PAGES 2+) ---
    print("-> Generating Page 2+: Detailed Domain Analysis...")
    analysis = report_data.get("Analysis", {})
    if not analysis:
        print("WARNING: 'Analysis' key is missing. Cannot generate the main report.")
        doc.build(story)
        return

    story.append(Paragraph("<b>Executive Summary</b>", styles['h2']))
    story.append(Paragraph(analysis.get('executive_summary', 'Not provided.'), styles['Normal']))
    add_space()

    story.append(Paragraph(f"<b>Overall Sentiment:</b> {analysis.get('overall_sentiment', 'N/A')}", styles['Normal']))
    add_space()

    story.append(Paragraph("<b>Key Domain Prominence (Ranked)</b>", styles['h2']))
    ranking = analysis.get('prominence_ranking', [])
    for i, item in enumerate(ranking):
        story.append(Paragraph(f"{i+1}. {item}", styles['Normal']))
    add_space(0.3)

    # --- Detailed Domain Analysis Loop ---
    domain_items = analysis.get('domain_analysis', [])
    if domain_items:
        story.append(MCLine(500))
        add_space(0.3)
        story.append(Paragraph("<b>Detailed Domain Analysis</b>", styles['h1']))
        add_space(0.1)

    for domain in domain_items:
        if not domain.get('is_present'):
            continue

        story.append(Paragraph(domain.get('domain', 'Unnamed Domain'), styles['h2']))
        add_space(0.1)

        story.append(Paragraph("<b>Summary of Impact:</b>", styles['h3']))
        story.append(Paragraph(domain.get('summary_of_impact', 'N/A'), styles['Normal']))
        add_space()

        story.append(Paragraph("<b>Key Illustrative Quote:</b>", styles['h3']))
        story.append(Paragraph(domain.get('key_illustrative_quote', 'N/A'), styles['Quote']))
        add_space()

        story.append(Paragraph("<b>Sub-theme Breakdown:</b>", styles['h3']))
        subthemes = domain.get('subtheme_analysis', [])
        if not subthemes:
            story.append(Paragraph("No sub-themes detected.", styles['Normal']))
        else:
            for subtheme in subthemes:
                story.append(Paragraph(subtheme.get('subtheme', 'N/A'), styles['SubTheme']))
                concepts = subtheme.get('detected_concepts', [])
                story.append(Paragraph("<i>Detected Concepts:</i> " + ", ".join(concepts), styles['Concepts']))
                add_space(0.1)

        add_space(0.3) # Add space after each full domain section

    # --- Build the PDF ---
    print("-> Assembling the final PDF document...")
    doc.build(story)
    print(f"-> PDF report successfully generated: {filename}")


# --- MAIN SCRIPT TO RUN THE GENERATION ---

print("STEP 1: Please upload your final analysis JSON file (e.g., 'analysis_of_Jackie_Barnes.json')."
uploaded_files = files.upload()

if not uploaded_files:
    print("\nNo file uploaded. Please run the cell again.")
else:
    input_filename = list(uploaded_files.keys())[0]
    print(f"\nSuccessfully uploaded: {input_filename}")

    try:
        # Step B: Read the analysis data from the uploaded file
        print("\nSTEP 2: Reading the analysis data from the file...")
        with open(input_filename, 'r', encoding='utf-8') as f:
            report_data_from_file = json.load(f)
        print("  -> Analysis data loaded successfully.")

        # Step C: Generate the PDF report using the new function
        print("\nSTEP 3: Generating the detailed PDF report...")
        patient_id = report_data_from_file.get("Patient_ID", "UnknownPatient")
        safe_patient_name = "".join(c for c in patient_id if c.isalnum() or c in (' ')).rstrip().replace(' ', '_')
        pdf_output_filename = f"Detailed_Report_for_{safe_patient_name}.pdf"

        # Call the new, specialized report generation function
        generate_detailed_report(report_data_from_file, pdf_output_filename)

        # Step D: Download the finished report
        print("\n--- Task Complete ---")
        files.download(pdf_output_filename)

    except json.JSONDecodeError:
        print(f"ERROR: The file '{input_filename}' is not a valid JSON file. Please check the content.")
    except Exception as e:
        print(f"An unexpected error occurred during PDF generation: {e}")

STEP 1: Please upload your final analysis JSON file (e.g., 'analysis_of_Jackie_Barnes.json').


Saving analysis_of_Jackie_Barnes (1).json to analysis_of_Jackie_Barnes (1).json

Successfully uploaded: analysis_of_Jackie_Barnes (1).json

STEP 2: Reading the analysis data from the file...
  -> Analysis data loaded successfully.

STEP 3: Generating the detailed PDF report...
-> Generating Page 1: Patient Demographics and History...
-> Generating Page 2+: Detailed Domain Analysis...
-> Assembling the final PDF document...
-> PDF report successfully generated: Detailed_Report_for_Jackie_Barnes.pdf

--- Task Complete ---


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# classification of emotions